## Imports

In [ ]:
import os
from glob import glob
from tqdm import tqdm

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy.integrate import simpson
from scipy.optimize import curve_fit
from scipy.signal import argrelextrema
from scipy.stats import spearmanr, median_abs_deviation, sigmaclip

from astropy.io import fits
from astropy.convolution import convolve, Gaussian1DKernel

%matplotlib inline

In [ ]:
plt.rcParams.update({'font.size': 20})

## Load Lines and Data

In [ ]:
data_path = '../data'
lines_vac, lines_air = np.loadtxt(f'{data_path}/hydroxyl_lines_rousselot_2000.txt').T

In [ ]:
results_path = '../results'
df = pd.read_csv(f'{results_path}/lines_norm_gdl_fit.csv', index_col=0)
columns = df.columns

In [ ]:
time_centered = df['spec_mjd'] - int(df['spec_mjd'].min())

In [ ]:
mask1_1 = (time_centered > 504) & (time_centered < 504.5)
mask1_2 = (time_centered > 504.58) & (time_centered < 505.515)
mask1_3 = (time_centered > 505.58) & (time_centered < 506.578)
mask1_4 = (time_centered > 506.61) & (time_centered < 507)
mask1 = mask1_1 | mask1_2 | mask1_3 | mask1_4
mask2 = (time_centered > 543) & (time_centered < 547)

In [ ]:
df_col_line_sum = [f'{line}_sum' for line in lines_vac]

In [ ]:
df_line_sums = df[df_col_line_sum][mask2]

## Correlation

In [ ]:
df_line_sums_pearson = df_line_sums.corr()
df_line_sums_spearman = df_line_sums.corr('spearman')

In [ ]:
line_sums_pearson = df_line_sums_pearson.values.copy()
line_sums_spearman = df_line_sums_spearman.values.copy()

In [ ]:
line_sums_pearson[line_sums_pearson<0] = 0
line_sums_spearman[line_sums_spearman<0] = 0

In [ ]:
fig, axs = plt.subplots(1,2,figsize=[20,10], dpi=200)
pcm_0 = axs[0].imshow(line_sums_pearson, cmap='inferno', origin='lower')
fig.colorbar(pcm_0, ax=axs[0], shrink=0.75)
pcm_1 = axs[1].imshow(line_sums_spearman, cmap='inferno', origin='lower')
fig.colorbar(pcm_1, ax=axs[1], shrink=0.75)
plt.tight_layout()
plt.show()

In [ ]:
lines_vac_xyc = []
for i in range(lines_vac.shape[0]):
    for j in range(lines_vac.shape[0]):
        element = [lines_vac[i]/10000, lines_vac[j]/10000, line_sums_pearson[i, j], line_sums_spearman[i, j]]
        lines_vac_xyc.append(element)

In [ ]:
lines_vac_xyc = np.array(lines_vac_xyc)

In [ ]:
fs = 40
plt.figure(figsize=[25,20], dpi=200)
plt.axes().set_facecolor('black')
plt.title('Pearson Correlation for OH Flux Time Series (Event 2)', fontsize=fs)
plt.scatter(lines_vac_xyc[:, 0], lines_vac_xyc[:, 1], c=lines_vac_xyc[:, 2], 
            s=4, alpha=1, marker='s', cmap='inferno')
plt.tick_params(labelsize=fs)
plt.xlabel('Wavelength ($\mu$m)', fontsize=fs)
plt.ylabel('Wavelength ($\mu$m)', fontsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)
plt.show()